In [95]:
%pwd

'c:\\Users\\Acer\\OneDrive\\Desktop'

In [98]:
import os
os.chdir(r"C:\\Users\\Acer\\OneDrive\\Desktop\\Skincarebot")
print(os.getcwd())

C:\Users\Acer\OneDrive\Desktop\Skincarebot


In [99]:
%pwd

'C:\\Users\\Acer\\OneDrive\\Desktop\\Skincarebot'

In [100]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [101]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [102]:
extracted_data=load_pdf_file(data='Data/')

In [103]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [104]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 3018


In [105]:
text_chunks


[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-03-15T13:18:55+00:00', 'title': 'Dataset Cleaning - Google Sheets', 'moddate': '2025-03-15T13:18:55+00:00', 'source': 'Data\\Dataset Cleaning - Google Sheets.pdf', 'total_pages': 584, 'page': 0, 'page_label': '1'}, page_content='Title Name of the Product Product Description\nMelaglow Rich Cream\nMelaglow Rich Cream is a powerful depigmentation and glow restoration creamdesigned to manage hyperpigmentation and brown spots. Formulated with potentingredients like arbutin, kojic acid, niacinamide, pTeroWhite, licorice extract, andglycolic acid, it smoothens, revitalizes, and brightens your skin, leaving it healthy andradiant.\nNew Follihair Tablet (Bottle)'),
 Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Geck

In [106]:
from langchain.embeddings import HuggingFaceEmbeddings
%pip install -U sentence-transformers

You should consider upgrading via the 'c:\Users\Acer\OneDrive\Desktop\Skincarebot\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [107]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [108]:
embeddings = download_hugging_face_embeddings()

In [109]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [110]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [111]:
import os
import pandas as pd
import faiss
import numpy as np
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from duckduckgo_search import DDGS 

In [112]:
# Convert text chunks to embeddings
text_vectors = [embeddings.embed_query(doc.page_content) for doc in text_chunks]
text_vectors = np.array(text_vectors, dtype='float32')

In [113]:
# Initialize FAISS index
dimension = text_vectors.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(text_vectors)  # Add embeddings to FAISS

In [114]:
# Store embeddings in FAISS
docsearch = FAISS.from_documents(text_chunks, embeddings)

In [115]:
# Set up retrieval
riever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [116]:
# Set up memory for conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [117]:
# Define system prompt
system_prompt = (
    "Hi, welcome to **Clinikally**! 😊 I am Mera, your skincare assistant. "
    "How can I help you today?\n\n"

    "You are an AI assistant for Skincare questions and skincare-related product recommendations. "
    "Use the retrieved context to answer questions, including correct product names, their use cases, "
    "suitable recommendations, and prices from your context database. "
    
    "If you do not get a sufficient answer from the database, search on DuckDuckGo for the latest information "
    "and provide an answer. "
    "suitability recommendations, and pricing from the database. "
    "If there is insufficient information in the database, search on open sources like DuckDuckGo "
    "and Clinikally.com to fetch relevant product details. "
    "When a user asks for a product recommendation, respond ONLY with the product name(s) from the knowledge base before the '|' symbol. "
    "Give details of the product as well when the user asks for the product names. "
    
    "Keep responses concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [118]:
# Define LLM
llm = OpenAI(temperature=0.4, max_tokens=500)

In [119]:
from langchain.chains import ConversationalRetrievalChain

# Create retrieval-based question-answering chain with memory
question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = ConversationalRetrievalChain.from_llm(
    retriever=riever,
    llm = llm,
    memory=memory
   
    
)

In [120]:
import requests
from bs4 import BeautifulSoup
%pip install beautifulsoup4

# Function to search for skincare products on Clinikally.com
def search_clinikally(query):
    search_url = f"https://www.clinikally.com/search?q={query.replace(' ', '+')}"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        results = []
        for product in soup.select(".product-card"):  # Adjust selector based on site structure
            title = product.select_one(".product-title").text.strip()
            price = product.select_one(".product-price").text.strip()
            link = "https://www.clinikally.com" + product.select_one("a")["href"]
            results.append(f"{title} - {price} [More Info]({link})")
        return "\n".join(results) if results else "No relevant products found."
    
    return "No relevant products found."

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Acer\OneDrive\Desktop\Skincarebot\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [121]:

# Function to search DuckDuckGo if FAISS doesn't have enough data
def search_duckduckgo(query):
    with DDGS() as ddgs:
        search_results = [result['body'] for result in ddgs.text(query, max_results=3)]
    return "\n".join(search_results) if search_results else "No relevant results found."


In [122]:
# Final function to answer user queries
def get_response(user_input):
    # Query FAISS first
    response = rag_chain.invoke({"question": user_input})
    answer = response["answer"]

    # If FAISS doesn't provide sufficient info, search Clinikally.com
    if "I don't know" in answer or len(answer.strip()) < 10:
        print("FAISS didn't provide enough information, searching Clinikally.com...")
        answer = search_clinikally(user_input)

        # If Clinikally also lacks results, fall back to DuckDuckGo
        if "No relevant products found." in answer:
            print("Clinikally.com didn't provide enough information, searching DuckDuckGo...")
            answer = search_duckduckgo(user_input)

    return answer

In [123]:
# Test responses
response1 = get_response("What are the best skincare products for acne?")
print(response1)

response2 = get_response("What is the best sunscreen for oily skin?")
print(response2)

response3 = get_response("What is the 3 best moisturizer for dry skin?") 
print(response3)

 Acnelak 4in1 Pimple Care Face Wash, Cuticapil Hair Serum Cream, Acorop Anti Acne Gel, and Palsons Renewderm Vit C Face Serum are all effective skincare products for acne.
 The S-Shield Sunscreen Lotion SPF 30 is the best sunscreen for oily skin as it has an oil-free, aqua-based texture and is enriched with vitamin E, Allantoin, and green tea extract to nourish the skin while providing broad-spectrum protection against UV-A and UV-B radiation.
 Trumist Moisturizing Lotion, Dermoﬂan AD Daily Emollient, and Moisturex Soft Lotion.


In [124]:
# Test responses with 10 relevant skincare-related questions
questions = [
    "What are the best skincare products for acne?",
    "What is the best sunscreen for oily skin?",
    "What are the 3 best moisturizers for dry skin?",
    "How can I treat dark circles under my eyes?",
    "What are some effective remedies for hyperpigmentation?",
    "Can you suggest a good face wash for sensitive skin?",
    "What ingredients should I avoid if I have acne-prone skin?",
    "How do I reduce redness on my face?",
    "What is the best serum for anti-aging?",
    "How do I build a basic skincare routine for combination skin?"
]

# Loop through the questions and get responses
for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question}")
    response = get_response(question)
    print("AI Response:", response, "\n")


Question 1: What are the best skincare products for acne?
AI Response:  Acorop Anti Acne Gel, Palsons Renewderm Vit C Face Serum, and Acnelak 4in1 Pimple Care Face Wash are all effective skincare products for acne. 

Question 2: What is the best sunscreen for oily skin?
AI Response:  IPCA Acne-UV Gel Sunscreen SPF 50/PA+ and Alblock Sunscreen Emulsion Gel SPF 50+ PA+++ are both suitable options for oily skin as they have a lightweight, non-greasy formula. Sunprotek Sunscreen Gel SPF 30+ PA+++ is also a good option for oily skin, but it may not be as lightweight as the other two options. Ultimately, it is important to choose a sunscreen with at least SPF 30 and broad-spectrum protection for optimal sun protection. 

Question 3: What are the 3 best moisturizers for dry skin?
AI Response:  Moisturex Soft Lotion, Dermaceutic C25 Cream, Trilogy Vital Moisturising Cream, and Aveeno Skin Relief Moisturizing Lotion are all effective moisturizers for dry skin. 

Question 4: How can I treat dark

In [125]:
print("Hi, welcome to **Clinikally**! 😊")
print("I am Mera, your skincare assistant. How can I help you today?\n")

while True:
    user_input = input("Ask a skincare-related question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        print("Goodbye! 😊 Stay glowing! ✨")
        break
    response = get_response(user_input)
    print("\nAI Response:", response, "\n")


Hi, welcome to **Clinikally**! 😊
I am Mera, your skincare assistant. How can I help you today?

Goodbye! 😊 Stay glowing! ✨
